## Imports

In [1]:
import boto3
import metrics
import numpy                 as np
import pandas                as pd
import matplotlib.pyplot     as plt
from sklearn.model_selection import train_test_split
from keras                   import regularizers
from keras.models            import Sequential
from keras.layers            import Dense, Flatten, Dropout
from keras.layers            import Conv2D, MaxPooling2D
from keras.utils             import np_utils
from IPython.core.display    import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline

# Declaring a random state for the modeling process

np.random.seed(42)

Using TensorFlow backend.


## Table Contents
     
 
- [Reading In The New Data](#Reading-In-The-New-Data)
    - [Data Overview](#Data-Overview)
    - [Mapping Labels](#Mapping-Labels)
    
    
- [Preprocessing](#Preprocessing)
    - [Declaring X and y](#Declaring-X-and-y)    
    - [Train-Test Splitting The Data](#Train-Test-Splitting-The-Data)
    - [Reshaping The Data](#Reshaping-The-Data)
    
    
- [Modeling](#Modeling)
    - [Setting Up The Neural Network](#Setting-Up-The-Neural-Network)
    - [Compiling The Neural Network](#Compiling-The-Neural-Network)
    - [Fitting The Training Data](#Fitting-The-Training-Data)
    - [Generating Predictions](#Generating-Predictions)
 
 
- [Evaluation](#Evaluation)

## Reading In The Data

In [2]:
# Reading in my credentials

with open("access_key", "r") as key:
    access_key = key.read()
    
with open("secret_key", "r") as secret:
    secret_access_key = secret.read()
    
# Declaring the client as an s3 bucket
    
client = boto3.client('s3', 
                      aws_access_key_id     = access_key,
                      aws_secret_access_key = secret_access_key)

# Naming the bucket and file

bucket_name = "google-quick-draw"
object_key  = "animals.csv"

# Creating a csv object

csv_obj = client.get_object(Bucket = bucket_name, 
                            Key    = object_key)

body = csv_obj['Body']

# Reading in the data

animals = pd.read_csv(body)

### Data Overview

In [3]:
# Checking the dimensions of the dataframe

animals.shape

(410123, 785)

In [4]:
# Looking at the head five rows

animals.head()

,label,0,1,2,3,4,5,6,7,8,...,774,775,776,777,778,779,780,781,782,783
0,bear,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,bear,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,bear,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,bear,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,bear,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
animals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410123 entries, 0 to 410122
Columns: 785 entries, label to 783
dtypes: int64(784), object(1)
memory usage: 2.4+ GB


In [6]:
# Looking for null values

animals.isnull().sum().sum()

0

### Mapping Labels

In [7]:
animals["label"].replace(to_replace ={"bear": 0,
                                      "cat" : 1,
                                      "dog" : 2},
                         inplace = True)

In [8]:
animals["label"].dtype

dtype('int64')

## Preprocessing

### Declaring X and y

In [9]:
# X variable

X = np.array(animals.drop(columns = "label")) 

# y variable

y = animals["label"].values

### Train-Test Splitting The Data

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state = 42, 
                                                    test_size    = 0.3)

In [11]:
# Scaling each X value

X_train = X_train.astype(float)
X_train /= 255

X_test  = X_test.astype(float)
X_test  /= 255

In [12]:
# Making sure the y variables are categorical

y_train = np_utils.to_categorical(y_train, 3)

y_test  = np_utils.to_categorical(y_test, 3)

### Reshaping The Data

In [13]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)

X_test  = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [14]:
# Checking the data type of the train and test sets

print(f"X_train's data type is {type(X_train)}")

print(f"X_test's data type is {type(X_test)}")

X_train's data type is <class 'numpy.ndarray'>
X_test's data type is <class 'numpy.ndarray'>


## Modeling

### Setting Up The Neural Network

#### Instantiating The Model

In [15]:
cnn = Sequential()

W1027 18:38:37.649209 140314990204736 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



#### Adding Layers

In [16]:
# Convolutional layers

# The first convolutional layer

cnn.add(Conv2D(filters     = 6,
               kernel_size = 3,
               activation  = "relu",
               input_shape = (28,28,1)))

# The first maxpooling layer

cnn.add(MaxPooling2D(pool_size = (2,2)))

# The second convolutional layer

cnn.add(Conv2D(128,
               kernel_size = 3,
               activation  = "relu"))

# The second maxpooling layer

cnn.add(MaxPooling2D(pool_size = (2,2)))


W1027 18:38:41.007398 140314990204736 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1027 18:38:41.011991 140314990204736 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1027 18:38:41.192208 140314990204736 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [17]:
# Dense layers

# Flattening the data

cnn.add(Flatten())

# Adding the first dense layer

cnn.add(Dense(128,
              activation         = "relu",
              kernel_regularizer = regularizers.l2(0.0001)))

# Adding the second dense layer

cnn.add(Dense(64,
              activation         = "relu",
              kernel_regularizer = regularizers.l2(0.0001)))

# Adding the third dense layer

cnn.add(Dense(32,
              activation         = "relu",
              kernel_regularizer = regularizers.l2(0.0001)))

# Adding the output layer: 3 because there are three classes

cnn.add(Dense(3,
              activation = "softmax"))

### Compiling The Neural Network

In [18]:
cnn.compile(loss      = "categorical_crossentropy",
            optimizer = "adam",
            metrics   = ["acc"])

W1027 18:39:12.499753 140314990204736 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1027 18:39:12.522532 140314990204736 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



### Fitting The Training Data

In [19]:
cnn.fit(X_train,
        y_train,
        validation_data = (X_train, y_train),
        batch_size      = 256,
        epochs          = 25,
        verbose         = 0)

W1027 18:39:20.332916 140314990204736 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1027 18:39:20.485534 140314990204736 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



In [20]:
# Training predictions

train_preds = cnn.predict_classes(X_train,
                                  batch_size = 100,
                                  verbose    = 0).ravel()

# Test predictions

test_preds = cnn.predict_classes(X_test,
                                 batch_size = 100,
                                 verbose    = 0).ravel()

In [21]:
# Evaluating y_train predictions

metrics.classification_table(y_train, train_preds)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass targets

[Top](#Table-Of-Contents)